In [1]:
import os
import numpy as np
import pandas as pd
import json
from copy import deepcopy
import re
import shutil
import random
import string
from random import choice
random.seed(10)
# -*- coding:utf-8 -*
import eval
from mkdir import mk_dir
from bert_format import bset,test_qa
import spokentonumer

In [11]:


col_ll = [
    ["样本量","num","How many patients are ivolved in this study?"],
    ["患者性别","gender","What's gender of the patient?"],
    ["年龄","age","How old is patient?"],
    ["证据等级","evi"],
    ["检测方式","mes"]]

for col_l in col_ll:
    
    col_name = col_l[0]
    col_ab = col_l[1]

    shutil.rmtree("eval/evl_%s" % (col_ab))
    mk_dir("eval/evl_%s" % (col_ab))

    # save 1-10
    train_set = json.load(open("input/input_%s/train_set.json" \
        %(col_ab), "r"))
    train_qa = train_set['data'][0]["paragraphs"]
    
    from sklearn.model_selection import KFold
    kf = KFold(n_splits = 10)
    re = 0
    for ttrain_qa,eval_qa in kf.split(train_qa):

        re = re + 1
        bset(
            [train_qa[i] for i in ttrain_qa],
            "eval/evl_%s/input_%s/" % (col_ab, re),
            "train_set.json")

        eval_q=[]
        eval_a={}
        for i in eval_qa:
            eval_q.append(test_qa(
                train_qa[i]['qas'][0]['id'],
                train_qa[i]['qas'][0]['question'],
                train_qa[i]['context']))
            eval_a[train_qa[i]['qas'][0]['id']] = train_qa[i]['qas'][0]['answers']

        bset(
            eval_q, 
            "eval/evl_%s/input_%s/" % (col_ab, re), 
            "test_set.json")
        json.dump(eval_a, open("eval/evl_%s/input_%s/eval_set.json" 
        % (col_ab, re), "w"))
    
    # 读取标注信息
    df = pd.read_excel("input/human_label.xlsx",index_col=0)
    df = df.loc[df["是否是致病菌"] == "yes",:]
    
    eval_q = []
    eval_a = {}
    for idx in df.index:

        if (df.loc[idx, "%s" %col_name] == "none"):
            
            if col_ab == "evi":
                "What kind of sample was used to isolated %s?" %df.loc[idx, "物种名称"]
            elif col_ab == "mes":
                que = "Which method was used to diagnose the infection of %s" %df.loc[idx, "物种名称"]
            else:
                que = col_l[2]
                    
            eval_q.append(
                test_qa(
                    str(idx), 
                    que,
                    df.loc[idx, "Full"].replace("\n",""))
            )
            eval_a[idx] = "none"

    bset(qa, "eval/evl_%s/input_11/" % (col_ab) ,"test_set.json" )
    json.dump(eval_a, open("eval/evl_%s/input_11/eval_set.json" % (col_ab), "w"))